In [3]:
! pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/93/21/9b035a4f823d6aee2917c75415be9a95861ff3d73a0a65e48edbf210cec1/tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata (3.6 kB)
Using cached tensorflow-2.15.0-cp311-cp311-win_amd64.whl (2.1 kB)


  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/93/21/9b035a4f823d6aee2917c75415be9a95861ff3d73a0a65e48edbf210cec1/tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.15.0 from https://files.pythonhosted.org/packages/4c/48/1a5a15517f18eaa4ff8d598b1c000300b20c1bb0e624539d702117a0c369/tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
  Obtaining dependency information for gast!=0.5.0,!=0.5.1,!=0.5.2,>=0.2.1 from https://files.pythonhosted.org/packages/fa

In [4]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

x_train = x_train.reshape(x_train.shape[0], 28*28)
x_val = x_val.reshape(x_val.shape[0], 28*28)
x_test = x_test.reshape(x_test.shape[0], 28*28)



4422102/4422102 [==============================] - 4s 1us/step


In [5]:
! pip install keras-tuner


  Obtaining dependency information for keras-tuner from https://files.pythonhosted.org/packages/2b/39/21f819fcda657c37519cf817ca1cd03a8a025262aad360876d2a971d38b3/keras_tuner-1.4.6-py3-none-any.whl.metadata
  Obtaining dependency information for kt-legacy from https://files.pythonhosted.org/packages/16/53/aca9f36da2516db008017db85a1f3cafaee0efc5fc7a25d94c909651792f/kt_legacy-1.0.5-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/128.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/128.9 kB ? eta -:--:--
   --- ------------------------------------ 10.2/128.9 kB ? eta -:--:--
   --------- ----------------------------- 30.7/128.9 kB 262.6 kB/s eta 0:00:01
   ------------ -------------------------- 41.0/128.9 kB 245.8 kB/s eta 0:00:01
   --------------------- ----------------- 71.7/128.9 kB 391.3 kB/s eta 0:00:01
   ------------------------------------ - 122.9/128.9 kB 554.9 kB/s eta 0:00:01
   ------------------------------------ - 122.9/128.9 kB

In [7]:
from tensorflow import keras
from tensorflow.keras.layers import Dense
from kerastuner.tuners import RandomSearch

def build_model(hp):
    model = keras.Sequential()
    
    model.add(keras.layers.Flatten(input_shape=(784,)))

    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(Dense(units=hp_units, activation='relu'))
    model.add(Dense(10, activation='softmax'))  # Output layer for 10 classes

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  
    executions_per_trial=1, 
    directory='my_dir',  
    project_name='fashion_mnist')

tuner.search_space_summary()

tuner.search(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))


Trial 10 Complete [00h 03m 22s]
val_accuracy: 0.8818333148956299

Best val_accuracy So Far: 0.8934166431427002
Total elapsed time: 00h 30m 11s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 384 and the optimal learning rate for the optimizer
is 0.001.

Epoch 1/10
1500/1500 [==============================] - 22s 14ms/step - loss: 0.4952 - accuracy: 0.8238 - val_loss: 0.4479 - val_accuracy: 0.8372
Epoch 2/10
1500/1500 [==============================] - 14s 10ms/step - loss: 0.3731 - accuracy: 0.8637 - val_loss: 0.3569 - val_accuracy: 0.8712
Epoch 3/10
1500/1500 [==============================] - 13s 9ms/step - loss: 0.3322 - accuracy: 0.8771 - val_loss: 0.3322 - val_accuracy: 0.8812
Epoch 4/10
1500/1500 [==============================] - 14s 9ms/step - loss: 0.3093 - accuracy: 0.8857 - val_loss: 0.3567 - val_accuracy: 0.8720
Epoch 5/10
1500/1500 [==============================] - 14s 9ms/step - loss: 0.2859 - accuracy: 0.8933 -